# Project: Web Scraping of hot stocks

### Siddharth Banerjee


**High Level Algorithm:**
<ol>
    <li>Read the code for the website "https://money.cnn.com/data/hotstocks/".</li>
    <li>Put each line from website code into list.</li>
    <li>Get the table data from the wbsite code and pass to ticker function.</li>
    <li>The ticker function compares each line from the table data with a regular expression to get ticker symbols.</li>
    <li>Read the code for the website "https://www.google.com/finance/quote/'+i+':NYSE" where 'i' is provided with ticker symbol.</li>
    <li>The "Previous close","Volume" and "PE Ratio" data is obtained from the website and stored in a data dictionary.</li>
    <li>The keys of the data dictionary are "Previous close","Volume" and "PE Ratio" and value being their corresponding values obtained from the website.</li>
    <li>The ticker symbols and the data in data dictionary are then put in "stocks.txt" file</li>
    <li>The data is then stored in "StocksTable" table in the "StocksDatabase" database.</li>
</ol>

**Instructions:** Please run the cells below.<br>
<br>
**Note: Since "Open" could not be found in Google finance website for each of the ticker symbols, "Previous close" has been used in place of that**
<br><br>
**Sample Output:**
<br><br>
Data saved successfully in stocks.txt<br>
Data saved successfully in StocksDatabase database


In [87]:
#Function returns ticker symbols for each line of table data

def ticker(line):
    expression = re.search('(<td.*>)(<a href.*>)(.*)(</a.*)(span.*)(</span.*)(</td>)', line)
    if expression:
        found = expression.group(3)
        #print(found)
        return found

In [88]:
#Function prints the ticker symbols along with their data into stocks.txt file

def filestore(dictionary1,list1):
    try:
        x='stocks.txt'
        dictionary2=dict()
        filehandle = open(x,'w')
        filehandle.write('Ticker Symbol')
        
        #The loop prints the headings into the file
        for i in list1:
            filehandle.write(', '+i)
        filehandle.write('\n')
        
        #The loop prints the data including the ticker symbols into the file
        for i in dictionary1:
            filehandle.write(i)
            dictionary2=dictionary1.get(i)
            for j in dictionary2:
                filehandle.write(', ')
                filehandle.write(str(dictionary2.get(j)))
            filehandle.write('\n')
        print('Data saved successfully in stocks.txt')
        filehandle.close()
    except:
        print('Error occurred while writing to text file')

In [89]:
#Printing to database
import sqlite3

def database_storage(dictionary2):
    try:
        dictionary1=dict()
        conn = sqlite3.connect('StocksDatabase.db')
        cur = conn.cursor()
        preclo=''
        volume=0
        PERatio=''
        cur.execute('DROP TABLE IF EXISTS "StocksTable"')
        cur.execute('CREATE TABLE StocksTable (Ticker Symb TEXT, PreviousClose REAL, Volume INTEGER, PERatio REAL)')
        for i in dictionary2:
            dictionary1=dictionary2.get(i)
            
            #data from the dictionary is stored in individual variables to be inserted into the database table.
            for j in dictionary1:
                if (j=='Previous close'):
                    preclo=dictionary1.get(j) #get the value for Previous close
                    preclo=preclo.replace('$','') #Replace $
                    preclo=float(preclo) #Convert value to float to save in DB table
                if (j=='Volume'):
                    volume=dictionary1.get(j) #get the value for Volume
                if (j=='P/E ratio'):
                    PERatio=dictionary1.get(j) #get the value for PE Ratio
                    PERatio=PERatio.replace('-','0') #Replace '-' with '0'
                    PERatio=float(PERatio)
            cur.execute('INSERT INTO StocksTable VALUES (?,?,?,?)',(i,preclo,volume,PERatio))
        conn.commit()
        print('Data saved successfully in StocksDatabase database')
        cur.close()
        conn.close()
    except:
        print('Error occurred while connecting to database')

In [90]:
#The function converts volume from a string into an integer value
def volume_converter(dictionary1):
    dictionary2=dict()
    for i in dictionary1:
        dictionary2=dictionary1.get(i)
        for j in dictionary2:
            if (j == 'Volume'):
                a=dictionary2.get(j)
                a=a.replace('M','')
                x=float(a)
                x=x*1000000.00
                x=int(x)
                dictionary2[j]=x
    return dictionary1

In [91]:
#The function copies data from one dictionary to the another
def copy_dictionary(dictionary2):
    dictionary1=dict()
    for i in dictionary2:
        dictionary1[i]=dictionary2.get(i)
    return dictionary1

In [92]:
#The function converts the data of Previous close, Volume and PE Ratio into a data dictionary
#The key for the data dictionary are Previous close, Volume and PE Ratio and the value being the value associated to each of them
def required_data(dictionary1,list1,list4):
    list2=list()
    dictionary2=dict()
    dictionary3=dict()
    for i in list1:
        list2=dictionary1.get(i)
        for j in list2:
            for k in list4:
                if (j==k):
                    s=list2.index(j)+2
                    dictionary2[j]=list2[s]
        dictionary3[i]=copy_dictionary(dictionary2)
        list2=list()
    return dictionary3

In [93]:
#The function reads the code of Google Finance Website and returns a dictionary with Previous close, Volume and PE Ratio data
from bs4 import BeautifulSoup
import requests

def databody(list1):
    try:
        dictionary1=dict()
        list2=list()
        for i in list1:
            website = requests.get('https://www.google.com/finance/quote/'+i+':NYSE').text
            soup = BeautifulSoup(website, "lxml")
            divlist = (soup.find_all("div", {"class": "gyFHrc"})) #To obtain from <div> tags having class as "gyFHrc"
            for div in divlist:
                a=div.find_all('div')
                for div in a:
                    list2.append(div.get_text(strip=False))
            dictionary1[i]=list2
            list2=list()
        return dictionary1
    except:
        print("There was a problem connecting to the website")

In [94]:
import urllib.request
import re

try:
    #Read the code from the website
    website=urllib.request.urlopen('https://money.cnn.com/data/hotstocks/')
    lines=website.readlines()
    list1=list()
    dictionary1=dict()
    dictionary2=dict()
    list4=['Previous close','Volume','P/E ratio']
    count=0
    #Put each line from website code into list
    for line in lines:
        count=count+1
        line=line.decode().strip()
        if (re.search('Most Actives',line)):
            break

    for line in range(count, len(lines)):
        a=lines[line].decode().strip()
        if (ticker(a) != None):
            list1.append(ticker(a))
        if (re.search('</table>',a)):
            break
    dictionary1=databody(list1)
    dictionary2=required_data(dictionary1,list1,list4)
    
    dictionary2=volume_converter(dictionary2)
    filestore(dictionary2,list4) #It calls the function to store data into Stock.txt
    database_storage(dictionary2) #It calls the function to store data in StockDatabase database
    
except:
    print('There was a problem connecting to the website')

Data saved successfully in stocks.txt
Data saved successfully in StocksDatabase database
